In [1]:
from flask import Flask, render_template, jsonify, request
from src.helper import download_hugging_face_embeddings
from langchain_pinecone import PineconeVectorStore
from langchain_mistralai import ChatMistralAI
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

# Initialize Flask app
app = Flask(__name__)

# Load environment variables
load_dotenv()

# Get API keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

# Ensure API keys exist
if not PINECONE_API_KEY or not MISTRAL_API_KEY:
    print("Missing API keys! Check your .env file.")
    exit(1)

# Download embeddings
embeddings = download_hugging_face_embeddings()

# Define Pinecone index name
index_name = "medicalbot"

# Load existing Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Initialize LLM
llm = ChatMistralAI(model="mistral-medium", temperature=0.4, max_output_tokens=500)

# Improved system prompt
system_prompt = (
    "You are a highly knowledgeable and reliable AI assistant specializing in medical advice. "
    "Your responses should be clear, accurate, and based on trusted medical sources. "
    "Follow these guidelines while answering: "
    "1] Fact-Based Responses: Use only the provided context and reliable knowledge. If unsure, say you don't know. "
    "2] Concise & Professional: Provide answers in a structured, easy-to-understand format (3-5 sentences max). "
    "3] Avoid Diagnosis: You are not a doctor. Provide general guidance but advise users to consult a professional for medical concerns. "
    "4] Patient Safety First: Warn users about risks when discussing treatments, medications, or serious conditions. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create RAG chain
from langchain.chains.combine_documents import create_stuff_documents_chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Flask Routes
@app.route("/")
def index():
    return render_template("chatbot.html")

@app.route("/get", methods=["POST"])
def chat():
    msg = request.form.get("msg", "").strip()

    if not msg:
        return jsonify("Empty message received!")

    print(f"User Input: {msg}")

    try:
        response = rag_chain.invoke({"input": msg})

        # Ensure response exists and contains a valid answer
        answer = response.get("answer", "").strip()

        if not answer or "error" in answer.lower():
            answer = "I'm sorry, but I don't know the answer to that."

        # Clean up response to remove unwanted characters
        answer = answer.replace("\n", " ").replace("\r", " ")
        answer = answer.replace("{", "").replace("}", "").replace('"', "")

        print(f"AI Response: {answer}")
        return jsonify(answer)

    except Exception as e:
        print(f"Error processing AI response: {str(e)}")
        return jsonify("An error occurred while fetching the response. Please try again.")

# Run Flask app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


c:\Users\aumpa\OneDrive\Desktop\Projects\Chatbot_Hackthon\Medical_Chatbot\src\helper.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.142.159:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2025 13:09:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 13:09:32] "GET /favicon.ico HTTP/1.1" 404 -


User Input: i have fever what to do?


127.0.0.1 - - [13/Mar/2025 13:09:51] "POST /get HTTP/1.1" 200 -


AI Response: If you have a fever, it's important to stay hydrated and rest. Over-the-counter medications like acetaminophen (Tylenol) or ibuprofen (Advil) can help reduce fever and alleviate discomfort, but they should not be given to children without consulting a healthcare provider. If your fever is very high, lasts for more than a few days, or is accompanied by other symptoms such as severe headache, stiff neck, confusion, or difficulty breathing, seek medical attention immediately, as these could be signs of a serious infection. However, please note that while fever is a common symptom of many illnesses, it is not a disease itself, and its underlying cause should be diagnosed and treated by a healthcare professional.
User Input: give me medication for fever


127.0.0.1 - - [13/Mar/2025 13:10:08] "POST /get HTTP/1.1" 200 -


AI Response: I'm an AI and cannot prescribe medication, but I can provide information. For fever management, over-the-counter options include acetaminophen (Tylenol) and ibuprofen (Advil). However, aspirin should not be given to children or adolescents due to the risk of Reye's syndrome. It's important to consult a healthcare professional for appropriate dosage and to ensure the fever is not a symptom of a more serious condition. Cool baths can also help alleviate high fevers. Remember, fevers are often the body's way of fighting infection, so they may not always need to be treated.
